In [116]:
import joblib
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import *

warnings.filterwarnings("ignore")

In [117]:
test_in = pd.read_csv('Database/test_pre_input.csv', index_col=0)
test_out = pd.read_csv('Database/test_pre_output.csv', index_col=0)

In [118]:
i = 0
name_list = ['None', 'SMOTE', 'Adasyn', 'KMeansSMOTE']
best_dt = joblib.load(f'Files/dt_{name_list[i]}_model.pkl')
best_lgb = joblib.load(f'Files/lgb_{name_list[i]}_model.pkl')
best_xgb = joblib.load(f'Files/xgb_{name_list[i]}_model.pkl')
best_cat = joblib.load(f'Files/cat_{name_list[i]}_model.pkl')

In [119]:
proba1 = best_dt.predict_proba(test_in)
proba2 = best_lgb.predict_proba(test_in)
proba3 = best_xgb.predict_proba(test_in)
proba4 = best_cat.predict_proba(test_in)
average_proba = (proba1+proba2+proba3+proba4)/4

dt_result = np.argmax(proba1, axis=1)
lgb_result = np.argmax(proba2, axis=1)
xgb_result = np.argmax(proba3, axis=1)
cat_result = np.argmax(proba4, axis=1)
soft_voting_result = np.argmax(average_proba, axis=1)

print(f"DT F1-Score:{f1_score(test_out, dt_result, average='micro'):.4f}")
print(f"lightGBM F1-Score:{f1_score(test_out, lgb_result, average='micro'):.4f}")
print(f"XGBoost F1-Score:{f1_score(test_out, xgb_result, average='micro'):.4f}")
print(f"CatBoost F1-Score:{f1_score(test_out, cat_result, average='micro'):.4f}")
print(f"Soft Voting F1-Score:{f1_score(test_out, soft_voting_result, average='micro'):.4f}")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
DT F1-Score:0.9459
lightGBM F1-Score:0.9424
XGBoost F1-Score:0.9414
CatBoost F1-Score:0.9283
Soft Voting F1-Score:0.9478
